In [1]:
from matplotlib import font_manager, rc  # 한글사용 matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import csv
import codecs
import numpy as np
import json
import tensorflow as tf
import jpype
from konlpy.tag import Okt
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame as df
InteractiveShell.ast_node_interactivity = 'all'
os.chdir("C:/Users/windd/pythondata")
%matplotlib inline
font_name = font_manager.FontProperties(
    fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
rdata = pd.read_csv('bigcontest/2019빅콘테스트_퓨처스리그2/AFSNT.csv',engine='python',encoding='cp949')

In [3]:
# 지연 여부 코딩

dly = []
for i in rdata.DLY:
    if i =='Y':
        dly.append(1)
    elif i =='N':
        dly.append(0)
rdata['DLY2'] = dly

In [4]:
yy = rdata.SDT_YY
mm = rdata.SDT_MM
dd = rdata.SDT_DD

# 날짜 병합 함수
from datetime import datetime

def time_merge(year,month,day):
    date = []
    for i in range(len(year)):
        date_str = str(year[i])+'-'+str(month[i])+'-'+str(day[i])
        date_date = datetime.strptime(date_str,'%Y-%m-%d')
        date.append(date_date)
    return date
a = time_merge(yy,mm,dd)

rdata['DATE'] = a

In [5]:
# 결항데이터 제거

rdata2 =rdata[~(rdata['CNL']=='Y')]
rdata2 = rdata2[~(rdata2.REG.isnull())]
rdata2.isnull().sum()

SDT_YY         0
SDT_MM         0
SDT_DD         0
SDT_DY         0
ARP            0
ODP            0
FLO            0
FLT            0
REG            0
AOD            0
IRR            0
STT            0
ATT            0
DLY            0
DRR       860509
CNL            0
CNR       979446
DLY2           0
DATE           0
dtype: int64

In [7]:
rdata3 = rdata2[['DATE','FLO','FLT','REG','ARP','ODP','AOD','IRR','DLY','STT','ATT','DLY2']]
rdata3 = rdata3.sort_values(by=['FLT','DATE'])
rdata_sorted = rdata3.sort_values(by=['DATE','FLO','FLT','REG'])
rdata_sorted.index = list(range(len(rdata_sorted)))
len(rdata_sorted)

979446

In [8]:
rdata_sorted.index = list(range(len(rdata_sorted)))

# 시간 범주화 함수 정의
def time_dicrete(stt):
    time_dic = []
    for i in range(len(stt)):
        st = int(stt[i].split(':')[0])*60 + int(stt[i].split(':')[1])
        if st >= 300 and st < 480:
            time_dic.append('5시-8시')
        elif st >= 480 and st < 660:
            time_dic.append('8시-11시')
        elif st >= 660 and st < 840:
            time_dic.append('11시-14시')
        elif st >= 840 and st < 1020:
            time_dic.append('14시-17시')
        elif st >= 1020 and st < 1200:
            time_dic.append('17시-20시')
        elif st >= 1200 and st < 1380:
            time_dic.append('20시-23시')
        elif st >=1380 or st<300:
            time_dic.append('23시-3시')
    return time_dic  

stt = list(rdata_sorted.STT)

a = time_dicrete(stt)

In [9]:
# 실제시각, 스케줄 시간 차이 계산 함수 정의

def time_difference(att,stt):
    time_dif = []
    for i in range(len(att)):
        at = int(att[i].split(':')[0])*60 + int(att[i].split(':')[1])
        st = int(stt[i].split(':')[0])*60 + int(stt[i].split(':')[1])
        time_dif.append(at-st)
    return time_dif

In [10]:
att = rdata_sorted.ATT
stt = rdata_sorted.STT

time_dif = time_difference(att,stt)

In [11]:
# 시간범주화, 시간차이 변수 추가

rdata_sorted['T_DISC'] = a
rdata_sorted['DIF'] = time_dif

In [12]:
# 일단 지연인데 시간차가 -로 많이 나는것들.. 일단은 그냥 지우고 시작 
# 지우는게 맞는거 같음

rdata_sorted[((rdata_sorted['DIF']<30) & (rdata_sorted['DLY2']==1))]
rdata_sorted = rdata_sorted[~((rdata_sorted['DIF']<30) & (rdata_sorted['DLY2']==1))]
len(rdata_sorted)

,DATE,FLO,FLT,REG,ARP,ODP,AOD,IRR,DLY,STT,ATT,DLY2,T_DISC,DIF
406103,2018-01-11,J,J1257,SEw3NzE5,ARP1,ARP3,A,N,Y,16:35,1:43,1,14시-17시,-892
406104,2018-01-11,J,J1257,SEw3NzE5,ARP3,ARP1,D,N,Y,15:25,0:56,1,14시-17시,-869
540840,2018-05-18,L,L1735,SEw4MjM1,ARP3,ARP1,A,N,Y,23:26,0:28,1,23시-3시,-1378


979443

# 현재는 출도착 맞지 않는 쌍이어도 진행했다.
# 이 부분은 확실치 않음
# 출도착 맞는 쌍으로만 진행하기도 했었음

In [18]:
# 출도착 쌍이 안맞긴함. 일단은 안 맞아도 그대로 진행했고
# 다른 버전은 출도착 맞는 쌍만 추출하여 진행했었다.


rdata_sorted.pivot_table(index='AOD',aggfunc='count',values='DLY')

# 생키다이어그램을 위한 전처리를 위해서 D만 생각한다.

rdata_d = rdata_sorted[rdata_sorted['AOD']=='D'].sort_values(['DATE','REG'])

,DLY
AOD,
A,489864
D,489579


In [19]:
# 항공기의 스케줄 시간을 수치화시키는 함수 정의
# 이를 통해 시간에 따라서 정렬할 수 있다.

rdata_d.index = list(range(len(rdata_d)))

def time(stt):
    time_dic = []
    for i in range(len(stt)):
        st = int(stt[i].split(':')[0])*60 + int(stt[i].split(':')[1])
        time_dic.append(st)
    return time_dic
time_dic = time(rdata_d.STT)

In [20]:
# 스케줄 시간 수치화 변수 할당

rdata_d['STT2'] = time_dic

In [21]:
# 날짜, 등록기호, 출발시간 순으로 보면 연결되었다는 것을 알 수 있다.

rdata_d = rdata_d.sort_values(['DATE','REG','STT2'])
rdata_d.index = list(range(len(rdata_d)))
rdata_d

,DATE,FLO,FLT,REG,ARP,ODP,AOD,IRR,DLY,STT,ATT,DLY2,T_DISC,DIF,STT2
0,2017-01-01,A,A1981,SEw3MjQ3,ARP1,ARP3,D,N,N,7:20,7:30,0,5시-8시,10,440
1,2017-01-01,A,A1954,SEw3MjQ3,ARP3,ARP1,D,N,Y,9:15,10:04,1,8시-11시,49,555
2,2017-01-01,A,A1983,SEw3MjQ3,ARP1,ARP3,D,N,Y,11:05,11:55,1,11시-14시,50,665
3,2017-01-01,A,A1984,SEw3MjQ3,ARP3,ARP1,D,N,Y,13:00,13:40,1,11시-14시,40,780
4,2017-01-01,A,A1985,SEw3MjQ3,ARP1,ARP3,D,N,Y,14:50,15:49,1,14시-17시,59,890
5,2017-01-01,A,A1934,SEw3MjQ3,ARP3,ARP1,D,N,Y,16:45,17:45,1,14시-17시,60,1005
6,2017-01-01,A,A1937,SEw3MjQ3,ARP1,ARP3,D,N,Y,18:35,19:38,1,17시-20시,63,1115
7,2017-01-01,A,A1978,SEw3MjQ3,ARP3,ARP1,D,N,Y,20:40,21:42,1,20시-23시,62,1240
8,2017-01-01,J,J1216,SEw3NDYw,ARP1,ARP3,D,N,N,9:55,10:13,0,8시-11시,18,595
9,2017-01-01,J,J1217,SEw3NDYw,ARP3,ARP1,D,N,N,12:10,12:25,0,11시-14시,15,730


In [22]:
# 출도착 같은 거 제거
# 공항1 -> 공항1 로 가는 데이터 있었음

rdata_d = rdata_d[~((rdata_d['ARP']=='ARP1') & (rdata_d['ODP']=='ARP1'))]
rdata_d = rdata_d.sort_values(['DATE','REG','STT2'])
rdata_d.index = list(range(len(rdata_d)))

In [24]:
# 전처리 이해 예제
# x는 하나의 등록기호가 하루의 얼마나 운행하는지 알 수 있고
# 하루에 운행된 것은 연결되었다고 판단
# y는 일별로 구분해야되기 때문에 하루에 운행하는 횟수를 리스트로 만들었다
# y를 참조해서 하루에 운행한 것들을 표현

x = rdata_d[rdata_d['REG']=='SEw3NzE4'][['DATE','REG','ARP','ODP','STT2']]
x = x.sort_values(['DATE','STT2'])
x.index = list(range(len(x)))
list1 = list(x.ARP+','+ x.ODP)
y = x.pivot_table(index='DATE',aggfunc='count',values='REG')
y = y.reset_index()
x

,DATE,REG,ARP,ODP,STT2
0,2017-01-01,SEw3NzE4,ARP6,ARP3,510
1,2017-01-01,SEw3NzE4,ARP3,ARP6,605
2,2017-01-01,SEw3NzE4,ARP6,ARP3,705
3,2017-01-01,SEw3NzE4,ARP3,ARP6,805
4,2017-01-01,SEw3NzE4,ARP6,ARP3,910
5,2017-01-01,SEw3NzE4,ARP3,ARP1,1040
6,2017-01-01,SEw3NzE4,ARP1,ARP3,1150
7,2017-01-01,SEw3NzE4,ARP3,ARP1,1255
8,2017-01-02,SEw3NzE4,ARP1,ARP2,540
9,2017-01-02,SEw3NzE4,ARP2,ARP1,630


In [25]:
# 최종적으로 데이터프레임으로 만들면 
# 각 행은 각 날짜에 운행한 것이며
# 열은 링크(엣지)로 생각하면 된다.

a=[]
b=0
qq= []

for i in range(len(y)):
    
    a = list1[b:y['REG'][i]+b]
    b += len(a)
    qq.append(a)
    
df = pd.DataFrame(qq)
df


,0,1,2,3,4,5,6,7,8,9
0,"ARP6,ARP3","ARP3,ARP6","ARP6,ARP3","ARP3,ARP6","ARP6,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None
1,"ARP1,ARP2","ARP2,ARP1","ARP1,ARP3","ARP3,ARP2",None,None,None,None,None,None
2,"ARP2,ARP3","ARP3,ARP2",None,None,None,None,None,None,None,None
3,"ARP2,ARP3","ARP3,ARP2","ARP2,ARP15","ARP15,ARP2",None,None,None,None,None,None
4,"ARP2,ARP1","ARP1,ARP3",None,None,None,None,None,None,None,None
5,"ARP3,ARP1","ARP1,ARP3","ARP3,ARP6","ARP6,ARP3","ARP3,ARP8","ARP8,ARP3","ARP3,ARP1",None,None,None
6,"ARP1,ARP3","ARP3,ARP8","ARP8,ARP3","ARP3,ARP1","ARP1,ARP2","ARP2,ARP3","ARP3,ARP2",None,None,None
7,"ARP2,ARP3","ARP3,ARP4","ARP4,ARP3","ARP3,ARP13","ARP13,ARP3","ARP3,ARP4","ARP4,ARP3","ARP3,ARP2",None,None
8,"ARP2,ARP15","ARP15,ARP2","ARP2,ARP3","ARP3,ARP2",None,None,None,None,None,None
9,"ARP2,ARP3","ARP3,ARP4","ARP4,ARP3","ARP3,ARP2","ARP2,ARP3","ARP3,ARP1",None,None,None,None


In [26]:
# 전처리 진행

rdata_d = rdata_d.sort_values(['DATE','REG','STT2'])
rdata_d.index = list(range(len(rdata_d)))
reg_list = list(rdata_d.REG.unique())
rdata_ddd = rdata_d[['DATE','REG','ARP','ODP','STT2']]

ex = []
for i in reg_list:
    x = rdata_ddd[rdata_ddd['REG']== i][['DATE','REG','ARP','ODP','STT2']]
    x = x.sort_values(['DATE','STT2'])
    x.index = list(range(len(x)))
    list1 = list(x.ARP+','+x.ODP)
    y = x.pivot_table(index='DATE',aggfunc='count',values='REG')
    y = y.reset_index()
    a = []
    qq = []
    b= 0 
    for j in range(len(y)):
        a = list1[b: y['REG'][j]+b]
        b += len(a)
        qq.append(a)
    ex.append(qq)

In [27]:
ss = sum(ex,[])
pdpd = pd.DataFrame(ss)
pdpd

,0,1,2,3,4,5,6,7,8,9,10,11
0,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None
1,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None
2,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None
3,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None
4,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None
5,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None
6,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None
7,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None
8,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None
9,"ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1","ARP1,ARP3","ARP3,ARP1",None,None,None,None


In [29]:
# 개수 맞음

a = 0
for i in range(12):
    a += pdpd[i].value_counts().sum()
a

len(rdata_ddd)

489574

489574

In [39]:
# 각 열은 링크 수준에서 (0, 1, 2 등) 출발공항에서 도착공항으로 몇 회 운행했는지 파악 가능

link_table = pd.DataFrame()
출발노드 = []
도착노드 = []
링크 = []
횟수 = []

for i in range(12):
    idx = pdpd[i].value_counts()

    for j in range(len(idx)):
        split_list = idx.index[j].split(',')
        출발노드.append(split_list[0])
        도착노드.append(split_list[1])
        링크.append(i)
        횟수.append(idx[j])

link_table['출발노드'] = 출발노드
link_table['도착노드'] = 도착노드
link_table['링크'] = 링크
link_table['횟수'] = 횟수

In [41]:
link_table2 = link_table.sort_values(by=['출발노드','도착노드'])
link_table2

,출발노드,도착노드,링크,횟수
64,ARP1,ARP10,0,1
160,ARP1,ARP10,2,2
19,ARP1,ARP11,0,597
89,ARP1,ARP11,1,288
148,ARP1,ARP11,2,41
191,ARP1,ARP11,3,69
229,ARP1,ARP11,4,466
278,ARP1,ARP11,5,189
327,ARP1,ARP11,6,58
15,ARP1,ARP12,0,874


In [42]:
# 중심성 분석 함수
# 가중치 고려

def get_degree_centrality(df,arp) :
    N = 15
    Z = df[(df['출발노드']==arp)|(df['도착노드']==arp)].횟수.sum()
    D = Z/(N-1)
    return D

In [43]:
# 중심성 분석 

arp_list = list(link_table2['출발노드'].unique())
center_link = pd.DataFrame(data=arp_list,columns=['공항'])
D_list = []

for i in arp_list:
    D = get_degree_centrality(link_table2, i)
    D_list.append(D)

center_link['중심성'] = D_list

center_link.sort_values('중심성',ascending = False)

,공항,중심성
8,ARP3,27912.785714
0,ARP1,21920.357143
7,ARP2,8598.428571
11,ARP6,2449.000000
9,ARP4,2385.857143
13,ARP8,2351.071429
10,ARP5,1138.642857
6,ARP15,1055.785714
14,ARP9,895.285714
3,ARP12,339.785714


In [44]:
# 가중치 고려 x

def degree_centrality(df,arp):
    a = df[df['출발노드']==arp]
    D = len((a['출발노드']+','+a['도착노드']).unique())/(N-1)
    return D

In [45]:
N =15
arp_list = list(link_table2['출발노드'].unique())
center= pd.DataFrame(data=arp_list,columns=['공항'])
D_list = []

for i in arp_list:
    D = degree_centrality(link_table2, i)
    D_list.append(D)
    
center['중심성'] = D_list
center

,공항,중심성
0,ARP1,1.000000
1,ARP10,0.214286
2,ARP11,0.071429
3,ARP12,0.214286
4,ARP13,0.142857
5,ARP14,0.142857
6,ARP15,0.285714
7,ARP2,0.642857
8,ARP3,0.857143
9,ARP4,0.357143


# 지연 네트워크 지연인것만으로 진행

In [46]:
rdata_d = rdata_d[rdata_d['DLY2']==1]

rdata_d = rdata_d.sort_values(['DATE','REG','STT2'])
rdata_d.index = list(range(len(rdata_d)))
reg_list = list(rdata_d.REG.unique())
rdata_ddd = rdata_d[['DATE','REG','ARP','ODP','STT2']]

ex = []
for i in reg_list:
    x = rdata_ddd[rdata_ddd['REG']== i][['DATE','REG','ARP','ODP','STT2']]
    x = x.sort_values(['DATE','STT2'])
    x.index = list(range(len(x)))
    list1 = list(x.ARP+','+x.ODP)
    y = x.pivot_table(index='DATE',aggfunc='count',values='REG')
    y = y.reset_index()
    a = []
    qq = []
    b= 0 
    for j in range(len(y)):
        a = list1[b: y['REG'][j]+b]
        b += len(a)
        qq.append(a)
    ex.append(qq)

In [47]:
ss = sum(ex,[])
pdpd = pd.DataFrame(ss)

In [48]:
# 개수 맞음

a = 0
for i in range(10):
    a += pdpd[i].value_counts().sum()
a

len(rdata_ddd)

85923

85923

In [50]:
link_table = pd.DataFrame()
출발노드 = []
도착노드 = []
링크 = []
횟수 = []

for i in range(10):
    idx = pdpd[i].value_counts()

    for j in range(len(idx)):
        split_list = idx.index[j].split(',')
        출발노드.append(split_list[0])
        도착노드.append(split_list[1])
        링크.append(i)
        횟수.append(idx[j])

link_table['출발노드'] = 출발노드
link_table['도착노드'] = 도착노드
link_table['링크'] = 링크
link_table['횟수'] = 횟수

In [51]:
link_table2 = link_table.sort_values(by=['출발노드','도착노드'])
link_table2

,출발노드,도착노드,링크,횟수
27,ARP1,ARP11,0,67
96,ARP1,ARP11,1,23
135,ARP1,ARP11,2,15
179,ARP1,ARP11,3,7
219,ARP1,ARP11,4,6
22,ARP1,ARP12,0,104
95,ARP1,ARP12,1,25
136,ARP1,ARP12,2,15
182,ARP1,ARP12,3,5
220,ARP1,ARP12,4,6


In [52]:
# 중심성 분석 

arp_list = list(link_table2['출발노드'].unique())
center_link_dly = pd.DataFrame(data=arp_list,columns=['공항'])
D_list = []

for i in arp_list:
    D = get_degree_centrality(link_table2, i)
    D_list.append(D)

center_link_dly['중심성'] = D_list

center_link_dly.sort_values('중심성',ascending = False)

,공항,중심성
8,ARP3,5333.357143
0,ARP1,3740.714286
7,ARP2,1403.571429
11,ARP6,458.000000
13,ARP8,428.571429
9,ARP4,386.714286
6,ARP15,168.214286
14,ARP9,99.071429
10,ARP5,90.214286
4,ARP13,69.000000


In [53]:
# 가중치 고려x

N =15
arp_list = list(link_table2['출발노드'].unique())
center_dly= pd.DataFrame(data=arp_list,columns=['공항'])
D_list = []

for i in arp_list:
    D = degree_centrality(link_table2, i)
    D_list.append(D)
    
center_dly['중심성'] = D_list
center_dly

,공항,중심성
0,ARP1,0.785714
1,ARP10,0.071429
2,ARP11,0.071429
3,ARP12,0.214286
4,ARP13,0.142857
5,ARP14,0.071429
6,ARP15,0.285714
7,ARP2,0.642857
8,ARP3,0.857143
9,ARP4,0.285714
